# Model with dit dah sequence recognition - 36 characters - element order prediction

Builds on `RNN-Morse-chars-single-ddp06` with element order encoding, In `RNN-Morse-chars-single-ddp06` particularly when applyijng minmax on the raw predictions we noticed that it was not good at all at sorting the dits and dahs (everything went to the dah senses) but was good at predicting the element (dit or dah) relative position (order). Here we exploit this feature exclusively. The length of dits and dahs is roughly respected and the element (the "on" keying) is reinforced from the noisy original signal.

Uses 5 element Morse encoding thus the 36 character alphabet.

Training material will be generated as random Morse strings rather than actual characters to maintain enough diversity

## Create string

Each character in the alphabet should happen a large enough number of times. As a rule of thumb we will take some multiple of the number of characters in the alphabet. If the multiplier is large enough the probability of each character appearance will be even over the alphabet. 

Seems to get better results looking at the gated graphs but procedural decision has to be tuned.

In [ ]:
import MorseGen

morse_gen = MorseGen.Morse()
alphabet = morse_gen.alphabet36
print(132/len(alphabet))

morse_cwss = MorseGen.get_morse_eles(nchars=132*5, nwords=27*5, max_elt=5)
print(alphabet)
print(len(morse_cwss), morse_cwss[0])

## Generate dataframe and extract envelope

In [ ]:
Fs = 8000
decim = 128
samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
n_prev = int((samples_per_dit/128)*12*2) + 1
print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/decim:.2f}. Look back is {n_prev}.')
label_df = morse_gen.encode_df_decim_ord_morse(morse_cwss, samples_per_dit, decim, 5)
env = label_df['env'].to_numpy()
print(type(env), len(env))

In [ ]:
import numpy as np

def get_new_data(morse_gen, SNR_dB=-23, nchars=132, nwords=27, morse_cwss=None, max_elt=5):
    decim = 128
    if not morse_cwss:
        morse_cwss = MorseGen.get_morse_eles(nchars=nchars, nwords=nwords, max_elt=max_elt)
    print(len(morse_cwss), morse_cwss[0])
    Fs = 8000
    samples_per_dit = morse_gen.nb_samples_per_dit(Fs, 13)
    #n_prev = int((samples_per_dit/decim)*19) + 1 # number of samples to look back is slightly more than a "O" a word space (3*4+7=19)
    #n_prev = int((samples_per_dit/decim)*23) + 1 # (4*4+7=23)
    n_prev = int((samples_per_dit/decim)*27) + 1 # number of samples to look back is slightly more than a "0" a word space (5*4+7=27)
    print(f'Samples per dit at {Fs} Hz is {samples_per_dit}. Decimation is {samples_per_dit/decim:.2f}. Look back is {n_prev}.')
    label_df = morse_gen.encode_df_decim_ord_morse(morse_cwss, samples_per_dit, decim, max_elt)
    # extract the envelope
    envelope = label_df['env'].to_numpy()
    # remove the envelope
    label_df.drop(columns=['env'], inplace=True)
    SNR_linear = 10.0**(SNR_dB/10.0)
    SNR_linear *= 256 # Apply original FFT
    print(f'Resulting SNR for original {SNR_dB} dB is {(10.0 * np.log10(SNR_linear)):.2f} dB')
    t = np.linspace(0, len(envelope)-1, len(envelope))
    power = np.sum(envelope**2)/len(envelope)
    noise_power = power/SNR_linear
    noise = np.sqrt(noise_power)*np.random.normal(0, 1, len(envelope))
    # noise = butter_lowpass_filter(raw_noise, 0.9, 3) # Noise is also filtered in the original setup from audio. This empirically simulates it
    signal = (envelope + noise)**2
    signal[signal > 1.0] = 1.0 # a bit crap ...
    return envelope, signal, label_df, n_prev

Try it...

In [ ]:
import matplotlib.pyplot as plt 

max_ele = morse_gen.max_ele(alphabet)
envelope, signal, label_df, n_prev = get_new_data(morse_gen, SNR_dB=-17, morse_cwss=morse_cwss, max_elt=max_ele)

# Show
print(n_prev)
print(type(signal), signal.shape)
print(type(label_df), label_df.shape)
    
x0 = 0
x1 = 1500

plt.figure(figsize=(50,4))
plt.plot(signal[x0:x1]*0.7, label="sig")
plt.plot(envelope[x0:x1]*0.9, label='env')
plt.plot(label_df[x0:x1].ele*0.9 + 1.0, label='ele')
plt.plot(label_df[x0:x1].chr*0.9 + 1.0, label='chr', color="orange")
plt.plot(label_df[x0:x1].wrd*0.9 + 1.0, label='wrd')
for i in range(max_ele):
    plt.plot(label_df[x0:x1][f'e{i}']*0.9 + 2.0 + i, label=f'e{i}')
plt.title("signal and labels")
plt.legend(loc=2)
plt.grid()

## Create data loader
### Define dataset

In [ ]:
import torch

class MorsekeyingDataset(torch.utils.data.Dataset):
    def __init__(self, morse_gen, device, SNR_dB=-23, nchars=132, nwords=27, morse_cwss=None, max_elt=5):
        self.max_ele = max_elt
        self.envelope, self.signal, self.label_df0, self.seq_len = get_new_data(morse_gen, SNR_dB=SNR_dB, morse_cwss=morse_cwss, max_elt=max_elt)
        self.label_df = self.label_df0
        self.X = torch.FloatTensor(self.signal).to(device)
        self.y = torch.FloatTensor(self.label_df.values).to(device)
        
    def __len__(self):
        return self.X.__len__() - self.seq_len

    def __getitem__(self, index):
        return (self.X[index:index+self.seq_len], self.y[index+self.seq_len])
    
    def get_envelope(self):
        return self.envelope
    
    def get_signal(self):
        return self.signal
    
    def get_X(self):
        return self.X
    
    def get_labels(self):
        return self.label_df
    
    def get_labels0(self):
        return self.label_df0
    
    def get_seq_len(self):
        return self.seq_len()
    
    def max_ele(self):
        return self.max_ele

### Define keying data loader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_chr_dataset = MorsekeyingDataset(morse_gen, device, -20, 132*5, 27*5, morse_cwss, max_ele)
train_chr_loader = torch.utils.data.DataLoader(train_chr_dataset, batch_size=1, shuffle=False) # Batch size must be 1

In [ ]:
signal = train_chr_dataset.get_signal()
envelope = train_chr_dataset.get_envelope()
label_df = train_chr_dataset.get_labels()
label_df0 = train_chr_dataset.get_labels0()

print(type(signal), signal.shape)
print(type(label_df), label_df.shape)

x0 = 0
x1 = 1500

plt.figure(figsize=(50,4))
plt.plot(signal[x0:x1]*0.8, label="sig", color="cornflowerblue")
plt.plot(envelope[x0:x1]*0.9, label='env', color="orange")
plt.plot(label_df[x0:x1].ele*0.9 + 1.0, label='ele', color="orange")
plt.plot(label_df[x0:x1].chr*0.9 + 1.0, label='chr', color="green")
plt.plot(label_df[x0:x1].wrd*0.9 + 1.0, label='wrd', color="red")
for i in range(max_ele):
    label_key = f'e{i}'
    plt.plot(label_df[x0:x1][label_key]*0.9 + 2.0, label=label_key)
plt.title("keying - signal and labels")
plt.legend(loc=2)
plt.grid()

## Create model classes

In [ ]:
import torch
import torch.nn as nn

class MorseLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )        
    
class MorseBatchedLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.input_size = input_size
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(1, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(1, 1, self.hidden_layer_size).to(self.device))
    
    def _minmax(self, x):
        x -= x.min(0)[0]
        x /= x.max(0)[0]
        
    def _hardmax(self, x):
        x /= x.sum()
        
    def _sqmax(self, x):
        x = x**2
        x /= x.sum()
        
    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(-1, 1, self.input_size), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        self._minmax(predictions[-1])
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(1, 1, self.hidden_layer_size).to(device),
            torch.zeros(1, 1, self.hidden_layer_size).to(device)
        )     
    
class MorseBatchedMultiLSTM(nn.Module):
    """
    Initial implementation
    """
    def __init__(self, device, input_size=1, hidden_layer1_size=6, output1_size=6, hidden_layer2_size=12, output_size=14):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.input_size = input_size
        self.hidden_layer1_size = hidden_layer1_size
        self.output1_size = output1_size
        self.hidden_layer2_size = hidden_layer2_size
        self.output_size = output_size
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_layer1_size)
        self.linear1 = nn.Linear(hidden_layer1_size, output1_size)
        self.hidden1_cell = (torch.zeros(1, 1, self.hidden_layer1_size).to(self.device),
                             torch.zeros(1, 1, self.hidden_layer1_size).to(self.device))
        self.lstm2 = nn.LSTM(input_size=output1_size, hidden_size=hidden_layer2_size)
        self.linear2 = nn.Linear(hidden_layer2_size, output_size)
        self.hidden2_cell = (torch.zeros(1, 1, self.hidden_layer2_size).to(self.device),
                             torch.zeros(1, 1, self.hidden_layer2_size).to(self.device))
    
    def _minmax(self, x):
        x -= x.min(0)[0]
        x /= x.max(0)[0]
        
    def _hardmax(self, x):
        x /= x.sum()
        
    def _sqmax(self, x):
        x = x**2
        x /= x.sum()
        
    def forward(self, input_seq):
        #print(len(input_seq), input_seq.shape, input_seq.view(-1, 1, 1).shape)
        lstm1_out, self.hidden1_cell = self.lstm1(input_seq.view(-1, 1, self.input_size), self.hidden1_cell)
        pred1 = self.linear1(lstm1_out.view(len(input_seq), -1))
        lstm2_out, self.hidden2_cell = self.lstm2(pred1.view(-1, 1, self.output1_size), self.hidden2_cell)
        predictions = self.linear2(lstm2_out.view(len(pred1), -1))
        self._minmax(predictions[-1])
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden1_cell = (
            torch.zeros(1, 1, self.hidden_layer1_size).to(device),
            torch.zeros(1, 1, self.hidden_layer1_size).to(device)
        )     
        self.hidden2_cell = (
            torch.zeros(1, 1, self.hidden_layer2_size).to(device),
            torch.zeros(1, 1, self.hidden_layer2_size).to(device)
        )         
    
class MorseLSTM2(nn.Module):
    """
    LSTM stack
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6, dropout=0.2):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size, num_layers=2, dropout=dropout)
        self.linear = nn.Linear(hidden_layer_size, output_size)
        self.hidden_cell = (torch.zeros(2, 1, self.hidden_layer_size).to(self.device),
                            torch.zeros(2, 1, self.hidden_layer_size).to(self.device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq), 1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
    def zero_hidden_cell(self):
        self.hidden_cell = (
            torch.zeros(2, 1, self.hidden_layer_size).to(device),
            torch.zeros(2, 1, self.hidden_layer_size).to(device)
        )        
        
class MorseNoHLSTM(nn.Module):
    """
    Do not keep hidden cell
    """
    def __init__(self, device, input_size=1, hidden_layer_size=8, output_size=6):
        super().__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_layer_size = hidden_layer_size
        self.lstm = nn.LSTM(input_size, hidden_layer_size)
        self.linear = nn.Linear(hidden_layer_size, output_size)

    def forward(self, input_seq):
        h0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        c0 = torch.zeros(1, 1, self.hidden_layer_size).to(self.device)
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1), (h0, c0))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]
    
class MorseBiLSTM(nn.Module):
    """
    Attempt Bidirectional LSTM: does not work
    """
    def __init__(self, device, input_size=1, hidden_size=12, num_layers=1, num_classes=6):
        super(MorseEnvBiLSTM, self).__init__()
        self.device = device # This is the only way to get things work properly with device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x.view(len(x), 1, -1), (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out[-1]    

Create the keying model instance and print the details

In [ ]:
#morse_chr_model = MorseBatchedMultiLSTM(device, hidden_layer1_size=12, output1_size=4, hidden_layer2_size=15, output_size=max_ele+3).to(device) # This is the only way to get things work properly with device
morse_chr_model = MorseBatchedLSTM(device, hidden_layer_size=50, output_size=max_ele+3).to(device)
morse_chr_loss_function = nn.MSELoss()
morse_chr_optimizer = torch.optim.Adam(morse_chr_model.parameters(), lr=0.002)
morse_chr_milestones = [4, 9]
morse_chr_scheduler = torch.optim.lr_scheduler.MultiStepLR(morse_chr_optimizer, milestones=morse_chr_milestones, gamma=0.5)

print(morse_chr_model)
print(morse_chr_model.device)

In [ ]:
# Input and hidden tensors are not at the same device, found input tensor at cuda:0 and hidden tensor at cpu
for m in morse_chr_model.parameters():
    print(m.shape, m.device)
X_t = torch.rand(n_prev)
X_t = X_t.cuda()
print("Input shape", X_t.shape, X_t.view(-1, 1, 1).shape)
print(X_t)
morse_chr_model(X_t)

In [ ]:
import torchinfo
torchinfo.summary(morse_chr_model)

## Train model

In [ ]:
it = iter(train_chr_loader)
X, y = next(it)
print(X.reshape(n_prev,1).shape, X[0].shape, y[0].shape)
print(X[0], y[0])
X, y = next(it)
print(X[0], y[0])

In [ ]:
%%time
from tqdm.notebook import tqdm

print(morse_chr_scheduler.last_epoch)
epochs = 4
morse_chr_model.train()

for i in range(epochs):
    train_losses = []
    loop = tqdm(enumerate(train_chr_loader), total=len(train_chr_loader), leave=True)
    for j, train in loop:
        X_train = train[0][0]
        y_train = train[1][0]
        morse_chr_optimizer.zero_grad()
        if morse_chr_model.__class__.__name__ in ["MorseLSTM", "MorseLSTM2", "MorseBatchedLSTM", "MorseBatchedLSTM2", "MorseBatchedMultiLSTM"]:
            morse_chr_model.zero_hidden_cell() # this model needs to reset the hidden cell
        y_pred = morse_chr_model(X_train)
        single_loss = morse_chr_loss_function(y_pred, y_train)
        single_loss.backward()
        morse_chr_optimizer.step()
        train_losses.append(single_loss.item())
        # update progress bar
        if j % 1000 == 0:
            loop.set_description(f"Epoch [{i+1}/{epochs}]")
            loop.set_postfix(loss=np.mean(train_losses))
    morse_chr_scheduler.step()

print(f'final: {i+1:3} epochs loss: {np.mean(train_losses):6.4f}')

In [ ]:
save_model = True
if save_model: 
    torch.save(morse_chr_model.state_dict(), 'models/morse_ord36_model')
else:
    morse_chr_model.load_state_dict(torch.load('models/morse_ord36_model', map_location=device))

<pre><code>
MorseBatchedLSTM(
  (lstm): LSTM(1, 50)
  (linear): Linear(in_features=50, out_features=8, bias=True)
)
final:  16 epochs loss: 0.0418
CPU times: user 35min 41s, sys: 39.3 s, total: 36min 20s
Wall time: 36min 10s
126 IFU DE F4EXB = R TNX RPT ES INFO ALEX = RIG IS FTD/1200 GW 10_W ANT IS 3AGI = WX IS SUNNY ES WARM 32C = HW AR F5SFU DE F4EXD K
final:   4 epochs loss: 0.0402
CPU times: user 8min 57s, sys: 9.29 s, total: 9min 7s
Wall time: 9min 4s
126 IFU DE F4EXB = R TNX RPT ES INFO ALE4 =RIG IS FTDXJ20_ PWR 100V ANT IS YAGI X WX IS SUNNY ES WARM 32C = HW AR R5SFU DE F4EXB K /// take 1 ///
126 IFU BE F4EKD = R TTX RPT ES ITFO ALEV XRIG IS FTDX1200 PWR 100V ANT ES YAGI = WX IS SUNNY EH PARM 32C = HW AR G5SFU DE U4EXB K /// take 2 ///
​final:   4 epochs loss: 0.0381 /// in fact it could be good to divide LR again by 2 at 20th epoch ///
CPU times: user 8min 55s, sys: 9.62 s, total: 9min 4s
Wall time: 9min 2s
127 IFU DE F4EXB = R TNX RPT ES INFO ALE4 = RIG IS FTDX1200 PWR 100V ANT IS YAGI X WX IS SUNNY ES WARM 32C = HW AR R5SFU DE F4EXB K
127 IFU TE F4EXD X R TTX RWT ES ITUÖ ALEV K NIM SI FTDX1200 PWR 100V ANT ES YAGI = WX IS IUNNY EH PARM 32C = HW AR G5SFU DE U4EXB K
final:   4 epochs loss: 0.0351
CPU times: user 8min 59s, sys: 9.74 s, total: 9min 9s
Wall time: 9min 6s
126 IFU DE F4EXB = R TNX RPT ES INFO ALE4 = RIG IS FTDX1200 PWR 100V ANT IS YAGI X WX IS SUNNY E GARM 32C = HW AR F5SFU DE F4EXB K
126 SFU NE F4EKB X R TTX RPT ES ITFO TLE4 = NIG SS FTDX1200 PWR 100V ANT ES YAGI X MX ES SUNNY E ÖARM Ü2C = HW AR F5SFU BE A4EXB K
final:   4 epochs loss: 0.0355
CPU times: user 9min 2s, sys: 10.1 s, total: 9min 12s
Wall time: 9min 10s
126 IFU DE F4EXB = R TNX RPT ES INFO ALEU = RIG IS FTDX12__ PWR 100V ANT IS YAGI X WX IS SUNNY E GARM V2C = HW AR F5SFU DE F4EXB K
126 IRU DE F4EXD X R TTX RPT ES INRO ALEW = GIM SS FTDX1200 PWR 100V ANT AS YAGI = MX IS IUNNY E ÖARM Ü2C = HW AR R5SFU DE U4EXB K
</code></pre>

In [ ]:
%%time
p_char_train = torch.empty(1,max_ele+3).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(train_chr_loader), total=len(train_chr_loader))
for j, train in loop:
    with torch.no_grad():
        X_chr = train[0][0]
        pred_val = morse_chr_model(X_chr)
        p_char_train = torch.cat([p_char_train, pred_val.reshape(1,max_ele+3)])        

In [ ]:
p_char_train = p_char_train[1:] # Remove garbge
print(p_char_train.shape) # t -> chars(t)

### Post process
  
  - Move to CPU to ger chars(time)
  - Transpose to get times(char)

In [ ]:
p_char_train_c = p_char_train.cpu() # t -> chars(t) on CPU
p_char_train_t = torch.transpose(p_char_train_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_char_train_c.shape, p_char_train_t.shape)

In [ ]:
X_train_chr = train_chr_dataset.X.cpu()
label_df_chr = train_chr_dataset.get_labels()
env_chr = train_chr_dataset.get_envelope()

l_alpha = label_df_chr[n_prev:].reset_index(drop=True)
plt.figure(figsize=(50,6))
plt.plot(l_alpha[x0:x1]["chr"]*3, label="ychr", alpha=0.2, color="black")
plt.plot(X_train_chr[x0+n_prev:x1+n_prev]*0.8, label='sig')
plt.plot(env_chr[x0+n_prev:x1+n_prev]*0.9, label='env')
plt.plot(p_char_train_t[0][x0:x1]*0.9 + 1.0, label='e', color="orange")
plt.plot(p_char_train_t[1][x0:x1]*0.9 + 1.0, label='c', color="green")
plt.plot(p_char_train_t[2][x0:x1]*0.9 + 1.0, label='w', color="red")
color_list = ["green", "red", "orange", "purple", "cornflowerblue"]
for i in range(max_ele):
    plt.plot(p_char_train_t[i+3][x0:x1]*0.9 + 2.0, label=f'e{i}', color=color_list[i])
plt.title("predictions")
plt.legend(loc=2)
plt.grid()

## Test

### Test dataset and data loader

In [ ]:
teststr = "F5SFU DE F4EXB = R TNX RPT ES INFO ALEX = RIG IS FTDX1200 PWR 100W ANT IS YAGI = WX IS SUNNY ES WARM 32C = HW AR F5SFU DE F4EXB KN"
test_cwss = morse_gen.cws_to_cwss(teststr)
test_chr_dataset = MorsekeyingDataset(morse_gen, device, -17, 132*5, 27*5, test_cwss, max_ele)
test_chr_loader = torch.utils.data.DataLoader(test_chr_dataset, batch_size=1, shuffle=False) # Batch size must be 1

### Run the model

In [ ]:
p_chr_test = torch.empty(1,max_ele+3).to(device)
morse_chr_model.eval()

loop = tqdm(enumerate(test_chr_loader), total=len(test_chr_loader))
for j, test in loop:
    with torch.no_grad():
        X_test = test[0]
        pred_val = morse_chr_model(X_test[0])
        p_chr_test = torch.cat([p_chr_test, pred_val.reshape(1,max_ele+3)])

In [ ]:
# drop first garbage sample
p_chr_test = p_chr_test[1:]
print(p_chr_test.shape)

In [ ]:
p_chr_test_c = p_chr_test.cpu() # t -> chars(t) on CPU
p_chr_test_t = torch.transpose(p_chr_test_c, 0, 1).cpu() # c -> times(c) on CPU
print(p_chr_test_c.shape, p_chr_test_t.shape)

### Show results

In [ ]:
X_test_chr = test_chr_dataset.X.cpu()
label_df_t = test_chr_dataset.get_labels()
env_test = test_chr_dataset.get_envelope()
l_alpha_t = label_df_t[n_prev:].reset_index(drop=True)

#### Raw results

In [ ]:
plt.figure(figsize=(100,4))
plt.plot(l_alpha_t[:]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:]*0.8, label='sig')
plt.plot(env_test[n_prev:]*0.9, label='env')
plt.plot(p_chr_test_t[0]*0.9 + 1.0, label='e', color="purple")
plt.plot(p_chr_test_t[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_chr_test_t[2]*0.9 + 2.0, label='w', color="red")
color_list = ["green", "red", "orange", "purple", "cornflowerblue"]
for i in range(max_ele):
    plt_a = plt.plot(p_chr_test_t[i+3]*0.9 + 3.0, label=f'e{i}', color=color_list[i])
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted.png')

### Integration by moving average

Implemented with convolution with a square window

In [ ]:
p_chr_test_tn = p_chr_test_t.numpy()
ele_len = round(samples_per_dit / 256)
win = np.ones(ele_len)/ele_len
p_chr_test_tlp = np.apply_along_axis(lambda m: np.convolve(m, win, mode='full'), axis=1, arr=p_chr_test_tn)

plt.figure(figsize=(100,4))
plt.plot(l_alpha_t[:]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:]*0.9, label='sig')
plt.plot(env_test[n_prev:]*0.9, label='env')
plt.plot(p_chr_test_tlp[0]*0.9 + 1.0, label='e', color="purple")
plt.plot(p_chr_test_tlp[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_chr_test_tlp[2]*0.9 + 2.0, label='w', color="red")
color_list = ["green", "red", "orange", "purple", "cornflowerblue"]
for i in range(max_ele):
    plt.plot(p_chr_test_tlp[i+3,:]*0.9 + 3.0, label=f'e{i}', color=color_list[i])
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted_lp.png')

### Apply threshold

In [ ]:
p_chr_test_tn = p_chr_test_t.numpy()
ele_len = round(samples_per_dit / 256)
win = np.ones(ele_len)/ele_len
p_chr_test_tlp = np.apply_along_axis(lambda m: np.convolve(m, win, mode='full'), axis=1, arr=p_chr_test_tn)

for i in range(max_ele+3):
    p_chr_test_tlp[i][p_chr_test_tlp[i] < 0.5] = 0

plt.figure(figsize=(100,4))
plt.plot(l_alpha_t[:]["chr"]*4, label="ychr", alpha=0.2, color="black")
plt.plot(X_test_chr[n_prev:]*0.9, label='sig')
plt.plot(env_test[n_prev:]*0.9, label='env')
plt.plot(p_chr_test_tlp[0]*0.9 + 1.0, label='e', color="purple")
plt.plot(p_chr_test_tlp[1]*0.9 + 2.0, label='c', color="green")
plt.plot(p_chr_test_tlp[2]*0.9 + 2.0, label='w', color="red")
color_list = ["green", "red", "orange", "purple", "cornflowerblue"]
for i in range(max_ele):
    plt.plot(p_chr_test_tlp[i+3,:]*0.9 + 3.0, label=f'e{i}', color=color_list[i])
plt.title("predictions")
plt.legend(loc=2)
plt.grid()
plt.savefig('img/predicted_lpthr.png')

## Procedural decision making

### take 1

Hard limits with hard values

In [ ]:
class MorseDecoderPos:
    def __init__(self, alphabet, dit_len, npos, thr):
        self.nb_alpha = len(alphabet)
        self.alphabet = alphabet
        self.dit_len = dit_len
        self.npos = npos
        self.thr = thr
        self.res = ""
        self.morsestr = ""
        self.pprev = 0
        self.wsep = False
        self.csep = False
        self.lcounts = [0 for x in range(3+self.npos)]
        self.morse_gen = MorseGen.Morse()
        self.revmorsecode = self.morse_gen.revmorsecode
        self.dit_l = 0.3
        self.dit_h = 1.2
        self.dah_l = 1.5
        print(self.dit_l*dit_len, self.dit_h*dit_len, self.dah_l*dit_len)
        
    def new_samples(self, samples):
        for i, s in enumerate(samples): # e, c, w, [pos]
            if s >= self.thr:
                self.lcounts[i] += 1
            else:
                if i == 1:
                    self.lcounts[1] = 0
                    self.csep = False
                if i == 2:
                    self.lcounts[2] = 0
                    self.wsep = False
            if i == 1 and self.lcounts[1] > 1.2*self.dit_len and not self.csep: # character separator
                morsestr = ""
                for ip in range(3,3+self.npos):
                    if self.lcounts[ip] >= self.dit_l*self.dit_len and self.lcounts[ip] < self.dit_h*self.dit_len: # dit
                        morsestr += "."
                    elif self.lcounts[ip] > self.dah_l*self.dit_len: # dah
                        morsestr += "-"
                char = self.revmorsecode.get(morsestr, '_') 
                self.res += char
                #print(self.lcounts[3:], morsestr, char)
                self.csep = True
                self.lcounts[3:] = self.npos*[0]
            if i == 2 and self.lcounts[2] > 2.5*self.dit_len and not self.wsep: # word separator
                self.res += " "
                #print("w")
                self.wsep = True


In [ ]:
dit_len = round(samples_per_dit / decim)
chr_len = round(samples_per_dit*2 / decim)
wrd_len = round(samples_per_dit*4 / decim)
print(dit_len)
decoder = MorseDecoderPos(alphabet, dit_len, max_ele, 0.9)
#p_chr_test_clp = torch.transpose(p_chr_test_tlp, 0, 1)
p_chr_test_clp = p_chr_test_tlp.transpose()
for s in p_chr_test_clp:
    decoder.new_samples(s) # e, c, w, [pos]
print(len(decoder.res), decoder.res)

### take 2

Hard limits with soft values

In [ ]:
class MorseDecoderPos:
    def __init__(self, alphabet, dit_len, npos, thr):
        self.nb_alpha = len(alphabet)
        self.alphabet = alphabet
        self.dit_len = dit_len
        self.npos = npos
        self.thr = thr
        self.res = ""
        self.morsestr = ""
        self.pprev = 0
        self.wsep = False
        self.csep = False
        self.scounts = [0 for x in range(3)] # separators
        self.ecounts = [0 for x in range(self.npos)] # Morse elements
        self.morse_gen = MorseGen.Morse()
        self.revmorsecode = self.morse_gen.revmorsecode
        self.dit_l = 0.3
        self.dit_h = 1.2
        self.dah_l = 1.5
        print(self.dit_l*dit_len, self.dit_h*dit_len, self.dah_l*dit_len)
        
    def new_samples(self, samples):
        for i, s in enumerate(samples): # e, c, w, [pos]
            if s >= self.thr:
                if i < 3:
                    self.scounts[i] += 1
            else:
                if i == 1:
                    self.scounts[1] = 0
                    self.csep = False
                if i == 2:
                    self.scounts[2] = 0
                    self.wsep = False
            if i >= 3:
                self.ecounts[i-3] += s
            if i == 1 and self.scounts[1] > 1.2*self.dit_len and not self.csep: # character separator
                morsestr = ""
                for ip in range(self.npos):
                    if self.ecounts[ip] >= self.dit_l*self.dit_len and self.ecounts[ip] < self.dit_h*self.dit_len: # dit
                        morsestr += "."
                    elif self.ecounts[ip] > self.dah_l*self.dit_len: # dah
                        morsestr += "-"
                char = self.revmorsecode.get(morsestr, '_') 
                self.res += char
                #print(self.ecounts, morsestr, char)
                self.csep = True
                self.ecounts = self.npos*[0]
            if i == 2 and self.scounts[2] > 2.5*self.dit_len and not self.wsep: # word separator
                self.res += " "
                #print("w")
                self.wsep = True


In [ ]:
dit_len = round(samples_per_dit / decim)
chr_len = round(samples_per_dit*2 / decim)
wrd_len = round(samples_per_dit*4 / decim)
print(dit_len)
decoder = MorseDecoderPos(alphabet, dit_len, max_ele, 0.9)
#p_chr_test_clp = torch.transpose(p_chr_test_tlp, 0, 1)
p_chr_test_clp = p_chr_test_tlp.transpose()
for s in p_chr_test_clp:
    decoder.new_samples(s) # e, c, w, [pos]
print(len(decoder.res), decoder.res)